# NTDS assignment 3: spectral graph theory
[Michaël Defferrard](http://deff.ch), *PhD student*, [EPFL](http://epfl.ch) [LTS2](http://lts2.epfl.ch)

The first two assignments were designed to warm you up. This third assignment is closer to what you'll have to do for the projects. It only misses the exploratory data analysis part (we'll do that later as an exercise). As such, this exercises is composed of two parts:
1. Data collection,
2. Data exploitation.

In [ ]:
%matplotlib inline

import configparser
import os

import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import scipy.sparse.linalg

plt.rcParams['figure.figsize'] = (17, 5)

## 1 Data collection

Like in any data project, the first part of the assignment is to collect some data. 

### 1.1 Get the genre of a single track

As often, we need an API key for certain operations. Add the following to your `credentials.ini` file. I gave a key during November 6th lab. If you were not there, ask one of your colleague.
```
[freemusicarchive]
api_key = MY-KEY
```

In [ ]:
# Read the confidential api key.
credentials = configparser.ConfigParser()
credentials.read(os.path.join('..', 'credentials.ini'))
api_key = credentials.get('freemusicarchive', 'api_key')

Your first task is to develop a function to retrieve the genre ID of a track given its track ID using the [FMA API](https://freemusicarchive.org/api).

Hints:
* A track might have multiple genres associated to it. Always return the first one and discard the others. Note: you should never discard data blindly. I selected the tracks so that this is not a problem.
* The `get_genre` function takes an integer as input, the track ID, and returns another integer, the genre ID.

In [ ]:
def get_genre(track_id):
    """Returns the genre of a track by querying the API."""
    BASE_URL = 'https://freemusicarchive.org/api/get/tracks.json'
    url = '{}?track_id={}&api_key={}'.format(BASE_URL, track_id, api_key)
    response = requests.get(url).json()
    return int(response['dataset'][0]['track_genres'][0]['genre_id'])

# A correct implementation should pass the below test.
assert get_genre(1219) == 38

### 1.2 Create a table of tracks

The `fma_tracks.csv` file contains a list of 2'000 tracks that we will use through this assignment. Each track belongs to either one of the following *top-level genres*: Electronic (`genre_id=12`) and Hip-Hop (`genre_id=21`).

In [ ]:
tracks = pd.read_csv('../data/fma_tracks.csv', index_col=0)
print('You got {} track IDs.'.format(len(tracks)))

Once imported by pandas, each row of the DataFrame represents a track.

In [ ]:
tracks.head(4)

### 1.3 Add the genre to the table

Your task is to add a `genre` column to the above created `tracks` DataFrame. The column should contain an integer which represents the genre of the track, i.e. the return value of the `get_genre` function you developed.

Hints:
* When developing, retrieve the genre of the first 10 tracks only. Only once your code works run it through all the tracks. That will save you time.
* As we want to apply one function (`get_genre`) to many data samples (the 2000 track IDs), use a functional approach. Check out `tracks.apply()`.
* In Python, you can declare an [anonymous function](https://en.wikipedia.org/wiki/Anonymous_function) as `lambda x: x + 1`.
* Your table should look like the below table, except with the correct number instead of 0.

In [ ]:
tracks['genre'] = 0
tracks.head(4)

In [ ]:
tracks = tracks[:3]

tracks['genre'] = tracks.apply(lambda track: get_genre(track.name), axis=1)

# Alternatively:
# tracks['genre'] = list(map(get_genre, tracks.index))

In [ ]:
tracks.head(4)

### 1.4 Save the data

To avoid having to collect the data everytime you restart the kernel, save the DataFrame as a CSV file.

In [ ]:
#tracks.to_csv('../data/fma_tracks_with_genre.csv')

You can now load it back with the following call instead of running the code in sections 1.1 to 1.3.

In [ ]:
tracks = pd.read_csv('../data/fma_tracks_with_genre.csv', index_col=0)

### 1.5 Data cleaning

As always, data cleaning is necessary when dealing with real (as opposed to synthetic) data. In this case, we only need to "summarize the genres". The tracks I've selected for the assignment are only part of the Electronic and Hip-Hop top genres. There *actual genre(s)* might however be more specific and be a sub-genre of those too. E.g. IDM is a sub-genre of Electronic. You can explore the genre hierarchy on the [Free Music Archive](http://freemusicarchive.org/genre/Electronic/).

In [ ]:
#tracks_m.loc[tracks.index, ('track', 'genres')].apply(lambda x: x[0]).unique()
#tracks['genre'] = tracks_m.loc[tracks.index, ('track', 'genres')].apply(lambda x: x[0])

In [ ]:
def get_top_genre(genre_id):
    return 21 if genre_id in [21, 83, 100, 539, 542, 811] else 12

tracks = tracks.applymap(lambda genre: get_top_genre(genre))
tracks.head(4)

If everything went fine, you should now have 1000 Rock (`genre_id=12`) and 1000 Hip-Hop (`genre_id=12`) tracks.

In [ ]:
tracks['genre'].value_counts()

## 2 Feature extraction

As is often the case, the data at hand is too large to be dealt with directly. We have to represent it with a smaller set of features, chosen to be maximally relevant to the task. (Manual feature extraction can sometimes be replaced by end-to-end learning systems.)

For music, MFCC are often relevant spectral features.

Listen to the music

Feature normalization

In [ ]:
features = pd.read_csv('../data/fma_features.csv', index_col=0, header=[0, 1, 2])
features = features.loc[tracks.index, 'mfcc']
assert (tracks.index == features.index).all()

features.shape, tracks.shape

In [ ]:
features -= features.mean(0)
features /= features.std(0)

## 3 Graph construction

* Is the graph connected?
* Shall we use the un-normalized or normalized Laplacian? Choose and justify.

Compute the l2. Or choose.

Hints:
* Use the `distance.pdist()` function.

### 3.1 Compute distances

Metric

The Euclidean distance is defined as $$d(i,j) = \|x_i - x_j\|_2$$

In [ ]:
from scipy.spatial import distance

distances = distance.pdist(features, metric='euclidean')
distances = distance.squareform(distances)

In [ ]:
plt.hist(distances.reshape(-1), bins=50);

Why are some distances equal to zero?

In [ ]:
print('{} distances equal exactly zero. Why?'.format(np.sum(distances == 0)))

### 3.2 Compute the weight matrix

Gaussian kernel $$\mathbf{W}(i,j) = \exp \left( \frac{-d^2(i, j)}{\sigma^2} \right)$$

In [ ]:
kernel_width = distances.mean()
weights = np.exp(-distances**2 / kernel_width**2)

np.fill_diagonal(weights, 0)

What kind of graph is that? Fully connected.

Sparsify the graph. Either knn or $\epsilon$. knn better to enforce connectedness.

In [ ]:
fix, axes = plt.subplots(2, 2, figsize=(17, 8))
def plot(weights, axes):
    axes[0].spy(weights)
    axes[1].hist(weights[weights > 0].reshape(-1), bins=50);
plot(weights, axes[:, 0])

if False:
    epsilon = np.percentile(weights, 80)
    weights[weights < epsilon] = 0
else:
    NEIGHBORS = 10
    idx = np.argsort(weights)[:, :-NEIGHBORS]
    for i in range(weights.shape[0]):
        weights[i, idx[i, :]] = 0
    weights = np.maximum(weights, weights.T)

plot(weights, axes[:, 1])

### 3.3 Compute the Laplacian

In [ ]:
degrees = weights.sum(0)

plt.hist(degrees, bins=50);

In [ ]:
# Combinatorial Laplacian.
laplacian = np.diag(degrees) - weights

# Normalized Laplacian.
deg_inv = np.diag(1 / np.sqrt(degrees))
laplacian = deg_inv @ laplacian @ deg_inv

# Alternatively:
# laplacian = np.identity(weights.shape[0]) - deg_inv @ weights @ deg_inv

plt.spy(laplacian)

In [ ]:
laplacian = sparse.csr_matrix(laplacian)

How many edges?

In [ ]:
print('{} edges out of {} x {} = {}'.format(laplacian.nnz, *weights.shape, weights.size))

### 3.4 Bonus

Can you think of a way to observe if the two genres form clusters in the graph we created?

Hint: Use only the weight matrix / laplacian and the labels.

Sort the rows and columns given the labels.

## 4 Eigenvectors & eigenvalues

No need to compute the Fourier basis, only the Fiedler vector, i.e. the eigenvector associated to $\lambda_2$.

Use one of the following functions: `np.linalg.eig`, `np.linalg.eigh`, `sparse.linalg.eigs`, `sparse.linalg.eigsh`. Justify your choice.

In [ ]:
eigenvalues, eigenvectors = sparse.linalg.eigsh(laplacian, k=10, which='SM')

# That's much slower:
# eigenvalues, eigenvectors = np.linalg.eigh(laplacian.toarray())

In [ ]:
plt.plot(eigenvalues, '.-');

Is the graph connected? Justify.

In [ ]:
eigenvalues

What do you expect as the result of the below computation? Justify. Do you get the value you expected? If not, why?

Note that `x @ y` is equivalent to `np.matmul(x, y)`. You should prefer the former as it makes it easier to read formulas.

In [ ]:
np.sum(laplacian @ eigenvectors[:, 0])

**Your answer here.** We expect zero because the first eigenvalue is zero. The small error is due to numerical precision.

## 5 Clustering

Compare different techniques.
PCA, Fiedler, spectral clustering

Visualization with Laplacian eigenmaps

Principal component analysis (PCA), no graph.

In [ ]:
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition

features_pca = skl.decomposition.PCA(n_components=2).fit_transform(features)

genres = skl.preprocessing.LabelEncoder().fit_transform(tracks)

plt.scatter(features_pca[:,0], features_pca[:,1], c=genres, cmap='RdBu', alpha=0.5);

In [ ]:
plt.scatter(eigenvectors[:, 1], eigenvectors[:, 2], c=genres, cmap='RdBu', alpha=0.5);

Cluster the tracks with the Fiedler vector. How many tracks were wrongly identified?

In [ ]:
labels = (eigenvectors[:, 1] > 0)

plt.scatter(eigenvectors[:, 1], eigenvectors[:, 2], c=labels, cmap='RdBu', alpha=0.5);

In [ ]:
err = np.sum(np.abs(labels - genres))
err = err if err < len(labels)/2 else len(labels) - err
print('{} errors ({}%)'.format(err, err/len(labels)*100))

Tune some parameters (e.g. `kernel_width`, `NEIGHBORS`) to get less errors. You should get an error rate lower than 15% (i.e. less than 300 errors in total). Try to understand the effect of each parameter.